# Spark logging demo

This walkthrough shows how to raise PySpark log verbosity so shuffle spilling and scheduler progress messages appear in the driver logs.

In [1]:
from spark_fuse.spark import create_session
from spark_fuse.utils.logging import enable_spark_logging

In [2]:
spark = create_session(app_name="spark-fuse-logging-demo")
spark

25/11/03 11:41:26 WARN Utils: Your hostname, MacBook-Air-von-Kevin.local resolves to a loopback address: 127.0.0.1; using 192.168.17.23 instead (on interface en0)
25/11/03 11:41:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/kevin/.ivy2/cache
The jars for the packages stored in: /Users/kevin/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d5af73cf-0916-4dce-8e81-d0b9de7a2f02;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 293ms :: artifacts dl 29ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0

In [3]:
enable_spark_logging(spark, level="INFO")

With logging enabled, execute a shuffle-heavy workload. Watch the driver output (stdout or log files) for messages like "Spilling partition" or "ExternalSorter" that indicate memory pressure.

In [4]:
df = spark.range(0, 1_000_000).repartition(16)
df.orderBy("id").count()

25/11/03 11:41:33 INFO CodeGenerator: Code generated in 320.724625 ms
25/11/03 11:41:33 INFO DAGScheduler: Registering RDD 4 (count at NativeMethodAccessorImpl.java:0) as input to shuffle 0
25/11/03 11:41:33 INFO DAGScheduler: Got map stage job 0 (count at NativeMethodAccessorImpl.java:0) with 2 output partitions
25/11/03 11:41:33 INFO DAGScheduler: Final stage: ShuffleMapStage 0 (count at NativeMethodAccessorImpl.java:0)
25/11/03 11:41:33 INFO DAGScheduler: Parents of final stage: List()
25/11/03 11:41:33 INFO DAGScheduler: Missing parents: List()
25/11/03 11:41:33 INFO DAGScheduler: Submitting ShuffleMapStage 0 (MapPartitionsRDD[4] at count at NativeMethodAccessorImpl.java:0), which has no missing parents
25/11/03 11:41:33 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 12.7 KiB, free 366.3 MiB)
25/11/03 11:41:34 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 6.0 KiB, free 366.3 MiB)
25/11/03 11:41:34 INFO BlockMana

1000000

In [5]:
spark.stop()

25/11/03 11:41:36 INFO SparkContext: SparkContext is stopping with exitCode 0.
25/11/03 11:41:36 INFO SparkUI: Stopped Spark web UI at http://192.168.17.23:4040
25/11/03 11:41:36 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
25/11/03 11:41:36 INFO MemoryStore: MemoryStore cleared
25/11/03 11:41:36 INFO BlockManager: BlockManager stopped
25/11/03 11:41:36 INFO BlockManagerMaster: BlockManagerMaster stopped
25/11/03 11:41:36 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
25/11/03 11:41:36 INFO SparkContext: Successfully stopped SparkContext
